In [1]:
import numpy as np
import pandas as pd
import datetime as dt
np.set_printoptions(precision=4, suppress=True)
MAX_NUM_OF_OUTCOME = 5

In [2]:
# rows are ifps and columns are users
fcasts = pd.read_csv('data0928/fcasts0928_with_ifp_turk_info.csv')
# Sort the fcasts by question and time, the previous fcasts will be flushed by the latest fcasts on the same ifp
fcasts = fcasts.sort_values(by=['timestamp', 'question_id', 'user_id'], ascending=[True, True, True])
#resolved_ifps = np.array(list(ifp_map))
#resolved_ifps = np.sort(resolved_ifps)
#print(resolved_ifps)

# team negtive: machine team
fcasts = fcasts[fcasts['team_id'] >= 0] #remove machine predictions
print("# of real human fcasts: ", len(fcasts))
#print("# of fcasts in total: ", len(fcasts))

ifps = pd.read_csv("data0928/ifps0928_refined.csv")
ifps = ifps.set_index('id')
#print(ifps.columns)
print("# of IFPs", len(ifps))
print(ifps.index)
ifps

/Users/heliaguin/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# of real human fcasts:  338292
# of IFPs 191
Int64Index([1065, 1029, 1101, 1092, 1038, 1056, 1074, 1114, 1047, 1117,
            ...
            2451, 2460, 2469, 2482, 2491, 2500, 2509, 2518, 2527, 2536],
           dtype='int64', name='id', length=191)


,Unnamed: 0,active?,answers,answers_count,binary?,brier_score,comments_count,ifp_created_at,description,discover_question_id,...,starts_at,type,updated_at,use_ordinal_scoring,void_reason,map_id,ground_truth,ifp_opens_at,ifp_closes_at,ifp_resolves_at
id,,,,,,,,,,,,,,,,,,,,,
1065,0,False,"[{'ended?': True, 'probability': 0.0, 'questio...",1,True,0.0,0,2018-03-14T09:30:14.627000,A “significant leadership disruption” includes...,210,...,2018-03-14T09:30:59,2,2018-09-04T20:52:04.321000,False,NaN,0,2.0,2018-03-14,2018-08-31,2018-09-04
1029,1,False,"[{'ended?': True, 'probability': 0.0, 'questio...",5,False,0.0,0,2018-03-14T09:30:08.108000,This question will be resolved using the last ...,189,...,2018-03-14T09:30:55,5,2018-07-17T13:27:39.727000,True,NaN,1,3.0,2018-03-14,2018-04-25,2018-04-26
1101,2,False,"[{'ended?': True, 'probability': 0.0, 'questio...",1,True,0.0,0,2018-03-14T09:30:22.205000,Article 7.1 enables the Council of the Europea...,215,...,2018-03-14T09:30:54,2,2018-09-10T10:49:22.093000,False,NaN,2,2.0,2018-03-14,2018-09-07,2018-09-10
1092,3,False,"[{'ended?': True, 'probability': 0.0, 'questio...",1,True,0.0,0,2018-03-14T09:30:20.682000,Temporary lifting of the blockade or partial l...,214,...,2018-03-14T09:30:53,2,2018-09-10T11:47:25.284000,False,NaN,3,2.0,2018-03-14,2018-09-07,2018-09-10
1038,4,False,"[{'ended?': True, 'probability': 1.0, 'questio...",3,False,0.0,0,2018-03-14T09:30:09.912000,This question will be resolved using monthly d...,207,...,2018-03-14T09:30:50,3,2018-07-17T13:27:39.821000,True,NaN,4,1.0,2018-03-14,2018-04-30,2018-06-05
1056,5,False,"[{'ended?': True, 'probability': 0.0, 'questio...",5,False,0.0,0,2018-03-14T09:30:13.460000,his question will be resolved using the Multis...,209,...,2018-03-14T09:30:49,5,2018-07-17T13:27:40.039000,True,NaN,5,4.0,2018-03-14,2018-03-16,2018-03-16
1074,6,False,"[{'ended?': True, 'probability': 0.0, 'questio...",1,True,0.0,0,2018-03-14T09:30:15.609000,A “significant leadership disruption” includes...,211,...,2018-03-14T09:30:49,2,2018-09-04T19:47:52.990000,False,NaN,6,2.0,2018-03-14,2018-08-31,2018-09-04
1114,7,False,"[{'ended?': True, 'probability': 0.0, 'questio...",5,False,0.0,0,2018-03-14T09:30:24.415000,This question will be resolved using the closi...,204,...,2018-03-14T09:30:17,5,2018-07-17T13:27:40.796000,True,NaN,7,4.0,2018-03-14,2018-03-29,2018-03-30
1047,8,False,"[{'ended?': True, 'probability': 0.0, 'questio...",3,False,0.0,0,2018-03-14T09:30:11.902000,EMPRES Global Animal Disease Information Syste...,208,...,2018-03-14T09:30:15,3,2018-07-17T13:27:39.903000,True,NaN,8,2.0,2018-03-14,2018-04-30,2018-05-15


In [3]:
# Processing the forecasts and generate preds, wager, conf and mse matrix
def generate_daily_pred_matrix(fcast_data, ifps, start_date):
    user_map = {}
    user_type_vector = []
    j = 0
    for i in fcast_data['user_id'].unique():
        user_map[i] = j
        team_id = fcast_data[fcast_data['user_id'] == i]['team_id'].iloc[0]
        if team_id == 1000: user_type_vector.append(3) # control group
        elif team_id < 100: user_type_vector.append(1) # volunteer
        else: user_type_vector.append(2) # turkers
        j += 1
    user_type_vector = np.array(user_type_vector)
    user_id_vector = fcast_data['user_id'].unique()
    m = len(user_map)
    n = len(ifps)
    print("matrix shape: ", n, m)
    
    #date = dt.datetime.strptime(fcast_data['created_at'].iloc[0][:10], "%Y-%m-%d").date()
    date = start_date
    print(date)
    indicator = np.zeros((n,m), dtype=int)
    #pred_matrix = -1 * np.ones((n,m), dtype=object)
    pred_matrix = -1 * np.ones((n,m, MAX_NUM_OF_OUTCOME))
    conf_matrix = -1 * np.ones((n,m))
    wager_matrix = -1 * np.ones((n,m))
    se_matrix = -1 * np.ones((n,m))
    preds_age_matrix = -1 * np.ones((n,m))
    open_ifps = np.zeros(n, dtype=int)
    
    daily_indicator = []
    daily_pred_matrix = []
    daily_conf_matrix = []
    daily_wager_matrix = []
    daily_se_matrix = []
    daily_open_vector = []
    daily_preds_age = []

    for idx, row in fcast_data.iterrows():
        while (row['timestamp'][:10]!=str(date)):  
            for qid, ifp in ifps.iterrows():
                map_id = ifp["map_id"]
                #print(qid, map_id)
                if str(date) >= ifp['ifp_opens_at'] and str(date) <= ifp['ifp_closes_at']:
                    open_ifps[map_id] += 1 # if an IFP is open, open_status open days so far
                elif str(date) >= ifp['ifp_resolves_at']:
                    open_ifps[map_id] = -2 # if an IFP is resolved, open_status=-2
                elif str(date) >= ifp['ifp_closes_at']:
                    open_ifps[map_id] = -1 # if an IFP is closed but not resolved, open_status=-1 
            #print(open_ifps)
            daily_indicator.append(np.array(indicator))
            daily_open_vector.append(np.array(open_ifps))
            daily_pred_matrix.append(np.array(pred_matrix))
            daily_conf_matrix.append(np.array(conf_matrix))
            daily_wager_matrix.append(np.array(wager_matrix))
            daily_se_matrix.append(np.array(se_matrix))
            daily_preds_age.append(np.array(preds_age_matrix))
            date += dt.timedelta(1)
            preds_age_matrix[preds_age_matrix>-0.1] += 1
            print(date)
            

        i = ifps.loc[row['question_id'], 'map_id']
        j = user_map[row['user_id']]
        indicator[i, j] = 1
        preds_age_matrix[i, j] = 0
        
        prediction = []
        ans_in_str = row['forecast']
        pos = ans_in_str.find("'value':")
        while pos >= 0:
            prediction.append(float(ans_in_str[pos+8: ans_in_str.find(',', pos)]))
            pos = ans_in_str.find("'value':", pos+1)
        if len(prediction)==1:
            prediction.append(100-prediction[0])
        prediction = np.array(prediction) / 100
        pred_matrix[i, j, :] = 0.0
        pred_matrix[i, j, :len(prediction)] = prediction

        conf_matrix[i, j] = row['confidence']
        wager_matrix[i, j] = row['wager']

        if np.isnan(row['ground_truth']): se_matrix[i, j] = -1
        else:
            true_label = int(row['ground_truth']-1)
            se_matrix[i, j] = np.sum(prediction**2) - prediction[true_label]**2 + (1-prediction[true_label])**2
    
    for qid, ifp in ifps.iterrows():
        map_id = ifp["map_id"]
        if str(date) >= ifp['ifp_opens_at'] and str(date) <= ifp['ifp_closes_at']:
            open_ifps[map_id] += 1 # if an IFP is open, open_status open days so far
        elif str(date) >= ifp['ifp_resolves_at']:
            open_ifps[map_id] = -2 # if an IFP is resolved, open_status=-2
        elif str(date) >= ifp['ifp_closes_at']:
            open_ifps[map_id] = -1 # if an IFP is closed but not resolved, open_status=-1 
    daily_indicator.append(np.array(indicator))
    daily_open_vector.append(np.array(open_ifps))
    daily_pred_matrix.append(np.array(pred_matrix))
    daily_conf_matrix.append(np.array(conf_matrix))
    daily_wager_matrix.append(np.array(wager_matrix))
    daily_se_matrix.append(np.array(se_matrix))
    daily_preds_age.append(np.array(preds_age_matrix))
    
    return (daily_indicator, daily_pred_matrix, daily_conf_matrix, daily_wager_matrix, daily_se_matrix,
            daily_open_vector, user_type_vector, user_id_vector, daily_preds_age)

daily_indicator, daily_pred_matrix, daily_conf_matrix, daily_wager_matrix, daily_se_matrix, daily_open_vector, user_type_vector, user_id_vector, daily_preds_age\
= generate_daily_pred_matrix(fcasts, ifps, dt.date(2018,3,7))

print(len(daily_pred_matrix))
print(daily_pred_matrix[0].shape)
#print(daily_pred_matrix[0])

matrix shape:  191 2353
2018-03-07
2018-03-08
2018-03-09
2018-03-10
2018-03-11
2018-03-12
2018-03-13
2018-03-14
2018-03-15
2018-03-16
2018-03-17
2018-03-18
2018-03-19
2018-03-20
2018-03-21
2018-03-22
2018-03-23
2018-03-24
2018-03-25
2018-03-26
2018-03-27
2018-03-28
2018-03-29
2018-03-30
2018-03-31
2018-04-01
2018-04-02
2018-04-03
2018-04-04
2018-04-05
2018-04-06
2018-04-07
2018-04-08
2018-04-09
2018-04-10
2018-04-11
2018-04-12
2018-04-13
2018-04-14
2018-04-15
2018-04-16
2018-04-17
2018-04-18
2018-04-19
2018-04-20
2018-04-21
2018-04-22
2018-04-23
2018-04-24
2018-04-25
2018-04-26
2018-04-27
2018-04-28
2018-04-29
2018-04-30
2018-05-01
2018-05-02
2018-05-03
2018-05-04
2018-05-05
2018-05-06
2018-05-07
2018-05-08
2018-05-09
2018-05-10
2018-05-11
2018-05-12
2018-05-13
2018-05-14
2018-05-15
2018-05-16
2018-05-17
2018-05-18
2018-05-19
2018-05-20
2018-05-21
2018-05-22
2018-05-23
2018-05-24
2018-05-25
2018-05-26
2018-05-27
2018-05-28
2018-05-29
2018-05-30
2018-05-31
2018-06-01
2018-06-02
2018-06-

In [19]:
# some sancity check
print('# of wager-option agents:')
print(np.sum(user_type_vector!=3))
for i, j in zip(daily_indicator, daily_pred_matrix):
    print(np.sum(i), np.sum(j[:, :, 0]!=-1))
# print whether all users have at least one prediction
print(np.sum(np.sum(daily_pred_matrix[-1][:, :, 0]!=-1, axis=0))>0)
print(not np.isnan(daily_se_matrix).any())

# of wager-option agents:
1178
228 228
733 733
1028 1028
1182 1182
1380 1380
1576 1576
1633 1633
1850 1850
2182 2182
2349 2349
2508 2508
2670 2670
2788 2788
2885 2885
3023 3023
3250 3250
3345 3345
3399 3399
3516 3516
3586 3586
3724 3724
3963 3963
4137 4137
4264 4264
4309 4309
4434 4434
4493 4493
4556 4556
4755 4755
4875 4875
4953 4953
5092 5092
5239 5239
5326 5326
5428 5428
5671 5671
5937 5937
6024 6024
6110 6110
6200 6200
6301 6301
6385 6385
6538 6538
6721 6721
6818 6818
6929 6929
7042 7042
7134 7134
7186 7186
7306 7306
7406 7406
7473 7473
7505 7505
7580 7580
7641 7641
7704 7704
7889 7889
9336 9336
10804 10804
11149 11149
11799 11799
12186 12186
12405 12405
14720 14720
15263 15263
15569 15569
15731 15731
16092 16092
16495 16495
16734 16734
18712 18712
19643 19643
20032 20032
20272 20272
20551 20551
20937 20937
21194 21194
23337 23337
23880 23880
24114 24114
24249 24249
24447 24447
24485 24485
25187 25187
27025 27025
27580 27580
27873 27873
28048 28048
28281 28281
28685 28685
28889 288

In [4]:
last_day_indicator = daily_indicator[-1]
## selecting users with more than a low upper number of answers
#min_answer_limit = 0
#valid = np.sum(last_day_indicator, axis=0)>=min_answer_limit
indicator = np.array(daily_indicator)#[:, :, valid]
preds = np.array(daily_pred_matrix)#[:, :, valid]
conf = np.array(daily_conf_matrix)#[:, :, valid]
wager = np.array(daily_wager_matrix)#[:, :, valid]
se = np.array(daily_se_matrix)#[:, :, valid]
truth = ifps['ground_truth'].values
openday = np.array(daily_open_vector)
user_type_vector = np.array(user_type_vector)#[valid]
user_id_vector = np.array(user_id_vector)#[valid]
preds_age = np.array(daily_preds_age)#[:, :, valid]

print(indicator.shape)
print(preds.shape)
print(conf.shape)
print(wager.shape)
print(se.shape)
print(truth.shape)
print(openday.shape)
print(user_type_vector.shape)
print(user_id_vector.shape)
print(preds_age.shape)
#print(np.sum(preds, axis=1))
save_file = open('data0928/daily_data_matrix_all_0928.npy','wb')
np.save(save_file, indicator)
np.save(save_file, preds)
np.save(save_file, conf)
np.save(save_file, wager)
np.save(save_file, se)
np.save(save_file, truth)
np.save(save_file, openday)
np.save(save_file, user_type_vector)
np.save(save_file, user_id_vector)
np.save(save_file, preds_age)
save_file.close()

#label_matrix = np.array(preds)
#label_matrix[preds >-0.1] = 2
#label_matrix[preds >=0.5] = 1
#np.savetxt(fname="label_matrix_HFC_bin_HRL.csv", X=label_matrix, delimiter=',', fmt="%d")
#np.savetxt(fname="preds_matrix_HFC_bin_HRL.csv", X=preds, delimiter=',', fmt="%.2f")
#np.savetxt(fname="truth_vector_HFC_bin_HRL.csv", X=truth, delimiter=',', fmt="%d")
#np.savetxt(fname="major_vector_HFC_mix_HRL.csv", X=major, delimiter=',', fmt="%d")

(185, 191, 2353)
(185, 191, 2353, 5)
(185, 191, 2353)
(185, 191, 2353)
(185, 191, 2353)
(191,)
(185, 191)
(2353,)
(2353,)
(185, 191, 2353)
